In [5]:
# NGSolve Libraries
from netgen.geom2d import unit_square
from ngsolve import *
from ngsolve.webgui import Draw # para jupyter
#import netgen.gui
from netgen.occ import *
from netgen.csg import *

import sys
sys.path.insert(0,"../")
import problems
import numpy as np
import params
import pickle
from time import time

In [6]:
## get problem parameters and geometry
problem = problems.problem3

phi0 = problem[0]['phi0']
chi = problem[0]['chi']
G_target = problem[0]['G']
geom = problem[1]
dim = problem[0]['dim']
BC = problem[2]
name = problem[-1]
h = 1
ord = 2
N = params.N
KBTV = params.KBTV
form = "Functional" # EDP //functional


phi = lambda J: phi0/J

G = Parameter(G_target)

## Generate mesh and geometry ### add parallel stuff
def mesher(geom, h):
    if ".stp" in geom:
        geo = OCCGeometry(geom)
    else:
        geo = pickle.load(open(geom, "rb"))

    mesh = Mesh(geo.GenerateMesh(maxh=h))
    return mesh
mesh = mesher(geom, h)  

def Gel_energy_functional(F):
    
    J = Det(F)
    phi = phi0/J
    H = (J - phi0)*log(1-phi)  + phi0 * chi*(1-phi) + phi0/1000*log(phi)
    return 0.5*(G/KBTV)* Trace(F.trans*F ) + H


## Generate spaces and forms
"""
To bond the gel go to geometries and describe the bonding
face there, not here.
"""
if BC["dir_cond"] == "faces":
    fesu = VectorH1(mesh, order=ord, dirichlet = BC["DIR_FACES"])
elif BC["dir_cond"] == "components":
    fesu = VectorH1(mesh, order=ord, dirichletx = BC["x"], dirichlety = BC["y"], dirichletz = BC["z"])

fesphi = L2(mesh, order = ord-2)

fes = FESpace([fesu, fesphi])
u, delta = fes.TrialFunction()
v, vphi= fes.TestFunction()
BF = BilinearForm(fes)


## Assemble forms
alpha = Parameter(1)

psih = GridFunction(fesphi)
psih.Set(0)

uk = GridFunction(fesu)
uk.Set((1,1,1))
psik = GridFunction(fesphi)
eps = 1e-6
# define phi as the 0 function 

gfu = GridFunction(fes)
gfu.vec[:] = 0
phi = CF(-z)    

def Assemble_BF(BF,G_funs,form):
    u, v, alpha, delta, psih, psik = G_funs
    F = Id(3)+ grad(u)
    if form == "Functional":
        BF += Variation(alpha * Gel_energy_functional(F).Compile()*dx) # add the "bilinear" form of specific problem

        return BF
    






In [7]:
""" WORKING PARAMETERS FOR SIMPLEST BONDED TEST"""
### 1 PHI = -Z
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 25

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*40 ,softening_n))

### 2

In [8]:
max_PG_it = 10
max_iter_qnewton = 20
max_iter_newton = 30
newton_damp = 0.5
softening_n = 30

tol_newton = 1e-6
tol_QN = 1e-6
tol_delta = 1e-6

gammas = np.flip(np.linspace(G_target, G_target*50 ,softening_n))


for num, load in enumerate(gammas):
    G.Set(load) #incremental softening
    if num == softening_n-1:
        max_PG_it = 20
        tol_newton = 1e-6
        tol_QN = 1e-8
        tol_delta = 1e-8
    for k in range(1,max_PG_it):

        print("PG it:",k)
        alpha.Set(2**k)
        psik.vec.data = psih.vec
        if k != 0:
            uk.vec.data = gfu.components[0].vec # update u_k to compare later
        print("Starting QN")
        for i in range(max_iter_qnewton):
            G_funs = u,v,alpha,delta,psih,psik 
            with TaskManager():
                BF = Assemble_BF(BF,G_funs,form) # get bi(not)linear form
            
            
            res = gfu.vec.CreateVector()
            w = gfu.vec.CreateVector()
            #iterations of newton for the NL problem at each QN step
            print("Solving nonlinear subproblem")
            for iter in range(max_iter_newton):
                with TaskManager():
                    BF.Apply(gfu.vec,res)
                    BF.AssembleLinearization(gfu.vec)
                    inv = BF.mat.Inverse(freedofs= fes.FreeDofs())
                    w.data = newton_damp * inv * res
                    gfu.vec.data += -w
                if abs(InnerProduct(w,res)) < tol_newton:
                    print("Tol achieved", InnerProduct(w,res))
                    break
            # check tolerance on QN
            qn_tol_val= Integrate((gfu.components[1])**2,mesh)
            if qn_tol_val < tol_delta:
                print("QN tol achieved",qn_tol_val )
                break
            # update psih
            psih.vec.data = psih.vec + gfu.components[1].vec
        # check error on gfu
        print("Error",Integrate((uk-gfu.components[0])**2,mesh))

    Draw(gfu.components[0],mesh)



PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved -8.389407956753591e-07
Solving nonlinear subproblem
Tol achieved -4.1110676994788134e-07
Solving nonlinear subproblem
Tol achieved -2.684070301282203e-07
Solving nonlinear subproblem
Tol achieved -5.916239294373125e-07
Solving nonlinear subproblem
Tol achieved -4.3075809392118247e-07
QN tol achieved 3.905109039905758e-08
Error 34.958960581165854
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 5.861652609226743e-07
Solving nonlinear subproblem
Tol achieved -5.565991385348146e-07
Solving nonlinear subproblem
Tol achieved -6.135931304948974e-07
Solving nonlinear subproblem
Tol achieved -8.058392095305825e-07
QN tol achieved 4.929536924470721e-08
Error 13.54726078889831
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 4.990177832722388e-07
Solving nonlinear subproblem
Tol achieved -2.624853232314901e-07
Solving nonlinear subproblem
Tol achieved -6.536535523757519e-07
Solving nonlinear subproblem
Tol

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 2.975421123334922e-07
Solving nonlinear subproblem
Tol achieved -2.790302483544072e-07
QN tol achieved 1.4137852251930304e-08
Error 0.006450899739929419
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 6.647430022020895e-07
Solving nonlinear subproblem
Tol achieved -4.649034213493261e-07
QN tol achieved 5.1795332653574766e-08
Error 0.001193709813161075
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 2.591654072312257e-07
Solving nonlinear subproblem
Tol achieved -5.784996927705196e-07
QN tol achieved 2.709283460708919e-08
Error 0.0012967902417537539
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 8.901817818720504e-07
Solving nonlinear subproblem
Tol achieved -3.396474657950442e-07
QN tol achieved 3.054461071488309e-08
Error 0.0007179177688766657
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 3.211525985334337e-07
Solving nonlinear subproblem
Tol achieved -3.029472

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 4.707685647496009e-07
Solving nonlinear subproblem
Tol achieved -4.402519546212534e-07
QN tol achieved 1.4400436335015935e-08
Error 0.007105752468030333
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 2.605635802810441e-07
Solving nonlinear subproblem
Tol achieved -7.279778104806324e-07
QN tol achieved 2.8796841774507404e-08
Error 0.001284849131058301
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 4.0645924638458596e-07
Solving nonlinear subproblem
Tol achieved -9.086875814829105e-07
QN tol achieved 2.889458562037097e-08
Error 0.0014295051163390117
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 3.5263196886167486e-07
Solving nonlinear subproblem
Tol achieved -5.440901178679023e-07
QN tol achieved 1.645842885393421e-08
Error 0.0008177995119453538
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 5.210079272649002e-07
Solving nonlinear subproblem
Tol achieved -4.9613

...

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 3.362618826917482e-07
Solving nonlinear subproblem
Tol achieved -3.9591838855245653e-07
QN tol achieved 6.795843374214492e-08
Error 18.10823434217087
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 2.5391095674708004e-07
Solving nonlinear subproblem
Tol achieved -7.349432770193142e-07
QN tol achieved 2.2040497650946233e-07
Error 0.11248814227339202
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 5.279651971751364e-07
Solving nonlinear subproblem
Tol achieved -4.355321891249969e-07
QN tol achieved 3.753178915194626e-07
Error 0.17648133589635961
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 3.3630551445841053e-07
Solving nonlinear subproblem
Tol achieved -2.7604812968382153e-07
Solving nonlinear subproblem
Tol achieved -2.566022061838876e-07
QN tol achieved 1.1731599319215838e-09
Error 0.3723086310458879
PG it: 5
Starting QN
Solving nonlinear subproblem
Tol achieved 7.73026673840

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

PG it: 1
Starting QN
Solving nonlinear subproblem
Tol achieved 3.8247835828764476e-07
Solving nonlinear subproblem
Tol achieved -6.636154771491822e-07
Solving nonlinear subproblem
Tol achieved -4.327149109929775e-07
QN tol achieved 9.971058891841619e-10
Error 154.1338403537001
PG it: 2
Starting QN
Solving nonlinear subproblem
Tol achieved 4.906062134273834e-07
Solving nonlinear subproblem
Tol achieved -3.8009059975040275e-07
Solving nonlinear subproblem
Tol achieved -5.987927575481665e-07
QN tol achieved 1.131303539181436e-09
Error 0.3342997658556658
PG it: 3
Starting QN
Solving nonlinear subproblem
Tol achieved 2.838416734716429e-07
Solving nonlinear subproblem
Tol achieved -9.196380445555746e-07
Solving nonlinear subproblem
Tol achieved -6.647310442489529e-07
QN tol achieved 1.5907668293898207e-09
Error 0.5401535448550278
PG it: 4
Starting QN
Solving nonlinear subproblem
Tol achieved 7.140183457795984e-07
Solving nonlinear subproblem
Tol achieved -6.08593558921865e-07
Solving nonline

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [9]:
# DO VTK    
vtk = VTKOutput(ma=mesh,coefs=[gfu.components[0],gfu.components[1]],names=["u","phi"],filename="gel")

vtk.Do()

'gel'